In [185]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [166]:
df=pd.read_csv('heart2.csv')

In [167]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [168]:
df.dtypes

Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object

In [169]:
df.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [170]:
df.ExerciseAngina.unique()

array(['N', 'Y'], dtype=object)

In [171]:
df.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [172]:
df.ChestPainType.unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [173]:
df['Sex']=df['Sex'].map({'M':1,'F':0})
df['RestingECG']=df['RestingECG'].map({'Normal':0,'ST':1,'LVH':2})
df['ExerciseAngina']=df['ExerciseAngina'].map({'Y':1,'N':0})
df['ST_Slope']=df['ST_Slope'].map({'Down':0,'Flat':1,'Up':2})

dummies = pd.get_dummies(df['ChestPainType'], prefix='ChestPainType', drop_first=True).astype(int)
df = pd.concat([df, dummies], axis=1)
df = df.drop('ChestPainType', axis=1)
df.head()

,Age,Sex,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,1,140,289,0,0,172,0,0.0,2,0,1,0,0
1,49,0,160,180,0,0,156,0,1.0,1,1,0,1,0
2,37,1,130,283,0,1,98,0,0.0,2,0,1,0,0
3,48,0,138,214,0,0,108,1,1.5,1,1,0,0,0
4,54,1,150,195,0,0,122,0,0.0,2,0,0,1,0


In [174]:
df.shape

(918, 14)

In [175]:
cts_col=['Cholesterol','MaxHR','Oldpeak','RestingBP']
for col in cts_col:
    df = df[(df[col] >= df[col].mean() - 3 * df[col].std()) & (df[col] <= df[col].mean() + 3 * df[col].std())]

In [176]:
df.shape

(899, 14)

In [177]:
X=df.drop("HeartDisease",axis=1)
y=df['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [178]:
model=LogisticRegression(max_iter=2000)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8555555555555555

In [179]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
model_rf.score(X_test, y_test)

0.8611111111111112

In [ ]:
model_svc=SVC()
model_svc.fit(X_train,y_train)
model_svc.score(X_test,y_test)

In [180]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

Retaining 95% data

In [181]:
pca=PCA(0.95)
X_train_pca=pca.fit_transform(X_train_scaled)
X_train_pca=pd.DataFrame(X_train_pca)
X_test_pca=pca.transform(X_test_scaled)

In [182]:
X_test_pca.shape,X_train_pca.shape

((180, 12), (719, 12))

In [183]:
model=LogisticRegression()
model.fit(X_train_pca,y_train)
model.score(X_test_pca,y_test)

0.8444444444444444

In [184]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train_pca, y_train)
model_rf.score(X_test_pca, y_test)

0.85

In [186]:
model_svc=SVC()
model_svc.fit(X_train_pca,y_train)
model_svc.score(X_test_pca,y_test)

0.8388888888888889

Accuracy decreased using PCA components as only 95% data was consider

Accuracy will decrease further if the dimensions are reduced further

That is the tradeoff need to be done with accuracy and dimensions